# Реализация AI агента для исследований по открытым источникам

In [1]:
from langchain_gigachat import GigaChat

## Для теста будут использоваться модели GigaChat lite, Pro и Max

In [65]:
import os
from dotenv import load_dotenv

load_dotenv()

giga_lite = GigaChat(
    credentials=os.getenv("GIGACHAT_APIKEY"),
    scope=os.getenv("GIGACHAT_SCOPE"),
    model="gigachat:latest",
    profanity_check=False,
    verify_ssl_certs=False,
)

giga_max = GigaChat(
    credentials=os.getenv("GIGACHAT_APIKEY"),
    scope=os.getenv("GIGACHAT_SCOPE"),
    model="GigaChat-Max",
    profanity_check=False,
    verify_ssl_certs=False,
)

giga_pro = GigaChat(
    credentials=os.getenv("GIGACHAT_APIKEY"),
    scope=os.getenv("GIGACHAT_SCOPE"),
    model="GigaChat-Pro",
    profanity_check=False,
    verify_ssl_certs=False,
)

## Создание агента планировщика поисковых запросов

In [3]:
QUERY_PLANNER_PROMPT = """Ты - агент планировщик поисковых запросов.
Твоя задача - анализировать задачу пользователя и разбить её на посиковые запросы для дпльнейшего исследования,
которое будет проводить другой агент.

Процесс работы
1. Анализ исходного запроса:
 - Определи основную тему и подтемы
 - Выявии ключевые сущности (персоны, организации, концепции)
 - Определи временные рамки (если важны)
 - Определи географический контекст (если нужен)
 - Определи тип информации (новости, исследования, мнения, данные)
 - Определи косвенные факторы влиающие на предмет исследования.

2. Генерация поисковых запросов:
 - Создай 3-6 отдельных поисковых запросов
 - Каждый запрос должен быть самодостаточным и конкретным
 - Используй естественный язык, но включай ключевые слова
 - Учитывай возможные синонимы и альтернативные формулировки
 - Разделяй мульти-аспектные запросы на отдельные направления

3. Принципы качества запросов:
 - Конкретность: избегай общих фраз
 - Релевантность: каждый запрос должен освещать отдельный аспект
 - Полнота: совокупность запросов должна покрывать все аспекты исходного вопроса
 - Эффективность: запросы должны быть сформулированы для максимальной релевантности в поисковых системах

Запрос пользователя:
{user_prompt}

Выведи только следующую информацию в формате JSON:
{format_instructions}
"""

In [4]:
USER_PROMPT = """**ИНФОРМАЦИЯ О МОЕЙ КОМПАНИИ:**

Компания «ДИО-Консалт» специализируется на внедрении комплексных корпоративных решений,
предоставляя услуги по автоматизации бизнес-процессов на основе программных продуктов «1С»,
независимо от сферы деятельности и от масштабов самого предприятия.

**Мой запрос:** Планируются переговоры с авиа-компанией 'Победа' (это наш потенциальный клиент),
мне нужно получить ключевые аргументы за то чтобы выбрали имено нас с побробным обоснованием.
"""

In [5]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field


class QueryPlannerResponse(BaseModel):
  original_query: str = Field(..., description="Исходный запрос пользователя")
  main_topic: str = Field(..., description="Основная тема")
  key_entities: list[str] = Field(
      default_factory=list,
      description="Ключевые сущности",
      examples=["сущность1", "сущность2"]
  )
  subtopics: list[str] = Field(default_factory=list, description="Подтемы")
  time_frame: str = Field(..., description="Актуальность/период")
  info_type: str = Field(..., description="Новости/исследования/мнения")
  search_strategy: str = Field(..., description="Краткое описание стратегии поиска")
  search_queries: list[str] = Field(
    default_factory=list, description="Поисковые запросы"
  )


parser = PydanticOutputParser(pydantic_object=QueryPlannerResponse)
query_planner_chain = (
    ChatPromptTemplate
    .from_messages([("system", QUERY_PLANNER_PROMPT)])
    .partial(format_instructions=parser.get_format_instructions())
    | giga_max
    | parser
)

Предварительный запуск агента

In [6]:
response = query_planner_chain.invoke({"user_prompt": USER_PROMPT})

response

QueryPlannerResponse(original_query="Планируются переговоры с авиа-компанией 'Победа', мне нужно получить ключевые аргументы за то чтобы выбрали именно нас с подробным обоснованием.", main_topic='Подготовка аргументов для переговоров', key_entities=['ДИО-Консалт', 'авиатранспортная компания Победа'], subtopics=['Услуги компании ДИО-Консалт', 'Опыт внедрения решений на предприятиях авиации', 'Преимущества автоматизации процессов авиакомпаний', 'Специализация компании на продуктах 1С', 'Отзывы клиентов использующих решения компании'], time_frame='Аргументация актуальна на текущее время', info_type='Исследования, обзоры, отзывы, кейсы', search_strategy='Изучение опыта успешных проектов аналогичных компаний, исследование потребностей и особенностей бизнеса авиакомпании "Победа", выявление конкурентных преимуществ наших услуг перед аналогичными предложениями конкурентов', search_queries=['опыт внедрения ИТ-решений в авиакомпаниях Россия', 'кейсы успешного использования ПО 1С в авиатранспорт

## Реализация поиска по открытым источникам

In [ ]:
from typing import Any

import requests


def search_in_google(query: str) -> list[dict[str, Any]]:
  url = "https://www.searchapi.io/api/v1/search?engine=google_light"
  headers = {"Authorization": f"Bearer {os.getenv('SEARCH_APIKEY')}"}
  params = {"q": query}
  response = requests.get(url, headers=headers, params=params)
  return response.json()["organic_results"]

In [10]:
from tqdm import tqdm

# Запрос -> результаты поиска
search_results: dict[str, list[dict]] = {}

for search_query in tqdm(response.search_queries):
    organic_results = search_in_google(search_query)
    search_results[search_query] = organic_results

100%|██████████| 6/6 [00:10<00:00,  1.74s/it]


In [13]:
k = 0
for results in search_results.values():
    k += len(results)
    
k

60

In [17]:
search_results

{'опыт внедрения ИТ-решений в авиакомпаниях Россия': [{'position': 1,
   'title': 'Персонализация, ИИ и свои решения: как идет технологическая ...',
   'link': 'https://www.gazeta.ru/business/2025/10/06/21811910.shtml',
   'displayed_link': 'www.gazeta.ru › business › 2025/10/06',
   'snippet': 'Так, руководство «Аэрофлота» приняло решение о создании дочерней компании «АФЛТ-Системс», чтобы вывести свои ИТ-проекты и весь российский ...',
   'date': 'Oct 6, 2025'},
  {'position': 2,
   'title': 'Авиакомпании и аэропорты внедряют российские ИТ-решения',
   'link': 'https://gazeta-don.ru/n932076.html',
   'displayed_link': 'gazeta-don.ru › ...',
   'snippet': 'Российское программное обеспечение внедрили более 60 аэропортов и десятки авиакомпаний, в числе которых есть и зарубежные.',
   'date': 'Sep 19, 2025'},
  {'position': 3,
   'title': 'ИТ-решения в гражданской авиации: исследование «РИВЦ ...',
   'link': 'https://rivc-pulkovo.ru/news/it-resheniya-v-grazhdanskoj-aviacii-issl/',
   'dis

In [14]:
def extract_text_by_url(url: str) -> str:
    api_url = "https://api.scraperapi.com/"
    payload = {
        "api_key": os.getenv("SCRAPPER_APIKEY"),
        "url": url,
        "output_format": "markdown"
    }
    response = requests.get(api_url, params=payload)
    return response.text

In [22]:
RANKER_PROMPT = """Твоя задача проанализировать задачу пользователя и поисковые результаты связанные с ней, после чего тебе нужно отранжировать результат на соотвествие задаче пользователя. Нужно оставить только те результаты, которые будут полезны для дальнейшего углубленого анализа.

**Задача пользователя**
{user_prompt}

**Результат поиска:**
{result}

**Выведи следующую информацию в формате JSON:**
{format_instructions}
"""

In [37]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    temperature=0, model_name="gpt-4o-mini", api_key=os.getenv("OPENAI_APIKEY")
)



class RankerResponse(BaseModel):
    rank: float = Field(
        ...,
        description="0 - результат не соотвествует задачи, 1 - очень полезен для анализа"
    )
    
ranker_parser = PydanticOutputParser(pydantic_object=RankerResponse)

ranker_chain = (
    ChatPromptTemplate
    .from_messages([("system", RANKER_PROMPT)])
    .partial(format_instructions=ranker_parser.get_format_instructions())
    | llm
    | ranker_parser
)

In [38]:
import json
import logging

logging.basicConfig(level=logging.INFO)

filtered_results = []
for query, results in tqdm(search_results.items()):
    for result in results:
        ranker_response = ranker_chain.invoke({
            "user_prompt": USER_PROMPT, "result": json.dumps(result, ensure_ascii=False)
        })
        if ranker_response.rank >= 0.5:
            filtered_results.append(result)

len(filtered_results)

  0%|          | 0/6 [00:00<?, ?it/s]INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
INFO:openai._base_client:Retrying request to /chat/completions in 20.000000 seconds
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
INFO:openai._base_client:Retrying request to /chat/completions in 20.000000 seconds
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
INFO:openai._base_clie

23

In [42]:
filtered_results

[{'position': 2,
  'title': 'Авиакомпании и аэропорты внедряют российские ИТ-решения',
  'link': 'https://gazeta-don.ru/n932076.html',
  'displayed_link': 'gazeta-don.ru › ...',
  'snippet': 'Российское программное обеспечение внедрили более 60 аэропортов и десятки авиакомпаний, в числе которых есть и зарубежные.',
  'date': 'Sep 19, 2025'},
 {'position': 3,
  'title': 'ИТ-решения в гражданской авиации: исследование «РИВЦ ...',
  'link': 'https://rivc-pulkovo.ru/news/it-resheniya-v-grazhdanskoj-aviacii-issl/',
  'displayed_link': 'rivc-pulkovo.ru › News',
  'snippet': 'Поставщик ИТ-решений для гражданской авиации «РИВЦ-Пулково» подготовил обзор текущего состояния и перспектив внедрения российских ИТ-разработок в авиационной ...'},
 {'position': 1,
  'title': 'Автоматизация учета авиадоходов - 1С:Проект года',
  'link': 'https://eawards.1c.ru/projects/avtomatizaciya-ucheta-aviadohodov-310438/',
  'displayed_link': 'eawards.1c.ru › projects › avtomatizaciya-ucheta-avia...',
  'snippet': 

In [59]:
RESEARCHER_PROMPT = """Роль и задача
Ты — Стратегический аналитик компании «ДИО-Консалт». Твоя задача — анализировать информацию о трендах ИТ в авиакомпаниях и преобразовывать её в убедительные аргументы для переговоров с потенциальными клиентами.

Контекст компании
«ДИО-Консалт» — эксперт по внедрению «1С» для автоматизации бизнес-процессов предприятий любого масштаба и сферы деятельности.

Процесс анализа
1. Анализ информационного контекста:

Выяви ключевые тренды ИТ в авиационной отрасли

Определи болевые точки и вызовы авиакомпаний

Найди примеры успешных внедрений и их результаты

Выяви потребности в импортозамещении и локальных решениях

2. Синтез аргументов для «ДИО-Консалт»:

Преобразуй отраслевые тренды в конкретные преимущества наших решений

Свяжи проблемы авиакомпаний с нашими компетенциями в 1С

Сформулируй измеримые выгоды для клиента

Подготовь ответы на возможные возражения

3. Адаптация под специфику клиента:

Учитывай особенности бизнес-модели (лоукостер, full-service и т.д.)

Предложи поэтапный подход к внедрению

Подготовь кейсы и аналогии из отрасли

**Страница для анализа**:
{page_url}
{page_text}

**Формат твоего ответа в формате Markdown (необязательно заполнять все поля, заполни только нужные для каждой страницы)**:

Источник: Ссылка на сайт
Заголовок: Заголовок страницы

Анализ отрасли:
 - ключевые_тренды
 - вызовы_авиакомпаний
 - успешные_кейсы
 
Аргументы для переговоров:
     - экономические:
         - аргумент: формулировка аргумента
         - обоснование: "подтверждение из данных
         - выгода: конкретная выгода для клиента
     - Технологические:
         - аргумент: формулировка аргумента
         - обоснование": подтверждение из данных
         - выгода: конкретная выгода для клиента
     - Операционные
         - аргумент: формулировка аргумента
         - обоснование: подтверждение из данных
         - выгода: конкретная выгода для клиента
   - Конкретные предложения:
      направление: область автоматизации
      решение: какое решение 1С предлагаем
      сроки: оценка внедрения
      результат: измеримый эффект
  
  Уникальные преимущества:
     - преимущество1 с привязкой к трендам
     - преимущество2 с привязкой к трендам

  Ответы на возражения:
      возражение: "типичное возражение клиента
      ответ: убедительный ответ с отсылкой к данным
"""

In [78]:
from langchain_core.output_parsers import StrOutputParser

researcher_chain = (
    ChatPromptTemplate.from_template(RESEARCHER_PROMPT)
    | giga_lite
    | StrOutputParser()
)

In [75]:
sites = []

for filtered_result in tqdm(filtered_results):
    text = extract_text_by_url(filtered_result["link"])
    sites.append({"url": filtered_result["link"], "text": text})

100%|██████████| 23/23 [03:09<00:00,  8.24s/it]


In [93]:
len(sites)

23

In [ ]:
researchs = []

logging.basicConfig(level=logging.INFO)

for site in tqdm(sites):
    response = researcher_chain.invoke({
        "page_url": site["url"], "page_text": site["text"]
    })
    researchs.append(response)